# Sobol SA

In [1]:
#### %matplotlib inline
from model import *
from agents import *
from globals import *
from server import *
from schedule import *
from utility import *
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner, BatchRunnerMP
import pathos
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import time

# We define our variables and bounds

fixed_params = {
    "network_type": 1
}

problem = {
    'num_vars': 3,
    'names': ['similarity_treshold', 'social_influence', 'swingers'],
    'bounds': [[0.01, 0.05], [0.01, 0.05], [1, 5]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 30
max_steps = 200
distinct_samples = 2

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples, calc_second_order = False)
# print(param_values)

# Set the outputs
model_reporters = {
#                 "preferences": compute_preferences,
                "percentage_majority_opinion": compute_opinions,
#                 "preference_A": compute_preference_A,
#                 "preference_B": compute_preference_B,
                "radical_opinions": compute_radical_opinions,
                "Echo_Chambers": echo_no,
                "Silent_Spiral": compute_silent_spiral
#                 "community_no": community_no,
}


# BatchRunner
batch = BatchRunnerMP(Network, 
                    max_steps=max_steps,
                    fixed_parameters=fixed_params,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters, nr_processes=8)
times = []
count = 0
for i in range(replicates):
    for vals in param_values:
        start = time.time()
        # Change parameters that should be integers
        vals = list(vals)
#         vals[2] = int(vals[2])

        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val
        variable_parameters['network_type']=1
        variable_parameters['malicious_N']=0
        variable_parameters['N']=1000
        variable_parameters['echo_threshold']=0.25
        variable_parameters['beta_component']=0.23
        variable_parameters['no_of_neighbors']= 3
        variable_parameters['swingers']=int(variable_parameters['swingers'])
#         print(variable_parameters)
#         print(variable_parameters)
        batch.run_iteration(variable_parameters, tuple(vals), count)
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')
        times.append(time.time() - start)
        meantime = np.mean(times)
        print('Average duration per iteration: %s seconds.'%str(meantime))
        nectime = ((len(param_values) * (replicates))-count)*meantime
        print('Expected time till finish: ', int(nectime/3600), 'hours and ', int((nectime%3600)/60), ' minutes.')
        
        
    
data = batch.get_model_vars_dataframe()
print(data)

100.00% done
Average duration per iteration: 78.9862312078476 seconds.
Expected time till finish:  0 hours and  0  minutes.
     similarity_treshold  social_influence  swingers  Run  Echo_Chambers  \
0               0.018789          0.013867  3.074219    0              5   
1               0.037070          0.013867  3.074219    1              2   
2               0.018789          0.021211  3.074219    2              6   
3               0.018789          0.013867  4.628906    3              4   
4               0.037070          0.021211  4.628906    4              5   
5               0.038789          0.033867  1.074219    5              5   
6               0.017070          0.033867  1.074219    6              6   
7               0.038789          0.041211  1.074219    7              8   
8               0.038789          0.033867  2.628906    8             10   
9               0.017070          0.041211  2.628906    9              4   
10              0.018789          0.0138

[300 rows x 9 columns]


In [2]:
#data.to_pickle('data1.pkl')
#data = pd.read_pickle('data.pkl')

In [3]:
def plot_index(s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)

    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

In [1]:
for Si in (radical_opinions,Echo_Chambers,Silent_Spiral,percentage_majority_opinion):
    # First order
    plot_index(Si, problem['names'], '1', 'First order sensitivity')
    plt.savefig('a')
    plt.show()

    # Second order
#     plot_index(Si, problem['names'], '2', 'Second order sensitivity')
#     plt.show()

    # Total order
    plot_index(Si, problem['names'], 'T', 'Total order sensitivity')
    plt.show()

NameError: name 'radical_opinions' is not defined